# SARSA와 Q-learning 응용

FrozenLake 환경에 SARSA와 Q-learning 을 적용해 보자.

### SARSA 프로그램 

FrozenLake-v0의 환경을 만들고 $Q(s,a) $를 key '(s,a)'를 가진 dictionary 데이터로 정의하고 초기치를 uniform(0,1)으로 부여하고 있다.

In [52]:
import gym
import random
env = gym.make('FrozenLake-v1')

Q= {}
for s in range(env.observation_space.n):
    for a in range(env.action_space.n):
        Q[(s,a)] = random.uniform(0,1)
        

다음은 epsilon-greedy policy 를 정의한다.

In [53]:
def epsilon_greedy(state,epsilon):
    if random.uniform(0,1) < epsilon:
        return env.action_space.sample()
    else:
        return max(list(range(env.action_space.n)),key=lambda x:Q[(state,x)])

Q(s,a)를 update하기 위해 학습률 $\alpha = 0.1$, $discount factor \gamma=0.9$, 그리고 epsilon-greedy policy를 위해 $\epsilon = 0.2$로 부여하고 있다. 프로그램에서 볼 수 있듯 $Q(s_{t+1},a_{t+1}) =Q(s\_,a\_)$라고 표시하였고 action a_의 선택은 epsilon-greedy policy로 결정하고있다.

In [54]:
alpha = 0.1
gamma = 0.9 # discount factor
epsilon = 0.2

num_episodes = 200000
timesteps = 1000

for i in range(num_episodes):
    s = env.reset()
    ### SARSA
    a = epsilon_greedy(s,epsilon)
    ###
    for t in range(timesteps):
        s_,r,done,_ = env.step(a)
        
        ###  SARSA
        a_ = epsilon_greedy(s_,epsilon)
        ###
        
        Q[(s,a)] = Q[(s,a)] + alpha*(r + gamma*Q[(s_,a_)] - Q[(s,a)])
        s = s_
        a = a_
        if done:
            break


아래 프로그램과 같이 최종 추정된 $Q(s,a)$를 출력하고 있으며

state 0에서 최적 policy는 action=1(down)

state 1 에서는 action=2(right)임을 보여주고 있다.


In [55]:
# SARSA

import pandas as pd
df = pd.DataFrame(list(Q.items()), columns=['state_action_pair', 'value'])
print(df.head(12))

   state_action_pair     value
0             (0, 0)  0.464199
1             (0, 1)  0.471603
2             (0, 2)  0.478725
3             (0, 3)  0.458124
4             (1, 0)  0.472069
5             (1, 1)  0.522923
6             (1, 2)  0.515306
7             (1, 3)  0.467932
8             (2, 0)  0.539115
9             (2, 1)  0.512157
10            (2, 2)  0.506227
11            (2, 3)  0.504466


아래 코드는 Q(14,1)이 가장 큰 값을 갖고 있으므로 state = 14에서 action=1 ( down)이 최적 policy인 것을 보여주고 있다. 


In [56]:
print([df[df['state_action_pair']==(14,0)]])
print([df[df['state_action_pair']==(14,1)]])
print([df[df['state_action_pair']==(14,2)]])
print([df[df['state_action_pair']==(14,3)]])

[   state_action_pair     value
56           (14, 0)  0.810836]
[   state_action_pair     value
57           (14, 1)  1.026793]
[   state_action_pair     value
58           (14, 2)  1.216296]
[   state_action_pair     value
59           (14, 3)  0.882546]


### Q-learning 실행 프로그램 

In [57]:
import gym
import random
import numpy as np
env = gym.make('FrozenLake-v1')

Q= {}
for s in range(env.observation_space.n):
    for a in range(env.action_space.n):
        Q[(s,a)] = random.uniform(0,1)
        

In [58]:
def epsilon_greedy(state,epsilon):
    if random.uniform(0,1) < epsilon:
        return env.action_space.sample()
    else:
        return max(list(range(env.action_space.n)),key=lambda x:Q[(state,x)])

In [59]:
alpha = 0.1
gamma = 0.9 # discount factor
epsilon = 0.2

num_episodes = 200000
timesteps = 1000

for i in range(num_episodes):
    s = env.reset()

    for t in range(timesteps):
        a = epsilon_greedy(s,epsilon)
        s_,r,done,_ = env.step(a)
        
        ### next Q의 action은 argmax로 결정 
        a_ = np.argmax([Q[(s,a)] for  a in range(env.action_space.n)])
        ###
        
        Q[(s,a)] = Q[(s,a)] + alpha*(r + gamma*Q[(s_,a_)] - Q[(s,a)])
        s = s_
        if done:
            break


In [60]:
import pandas as pd
df = pd.DataFrame(list(Q.items()), columns=['state_action_pair','value'])
print(df.head(12))


   state_action_pair     value
0             (0, 0)  0.456220
1             (0, 1)  0.484444
2             (0, 2)  0.457622
3             (0, 3)  0.454794
4             (1, 0)  0.502102
5             (1, 1)  0.576188
6             (1, 2)  0.477732
7             (1, 3)  0.460257
8             (2, 0)  0.475832
9             (2, 1)  0.484957
10            (2, 2)  0.476598
11            (2, 3)  0.454844


아래 프로그램은 Q(14,1)이 가장 큰 값을 갖고 있으며 state=14에서 action = 1이 최적 policy임을 보여주며, SARSA와 다른 결과를 보여주고있다.

In [61]:
print([df[df['state_action_pair']==(14,0)]])
print([df[df['state_action_pair']==(14,1)]])
print([df[df['state_action_pair']==(14,2)]])
print([df[df['state_action_pair']==(14,3)]])

[   state_action_pair     value
56           (14, 0)  0.791015]
[   state_action_pair     value
57           (14, 1)  1.294372]
[   state_action_pair     value
58           (14, 2)  0.985858]
[   state_action_pair     value
59           (14, 3)  0.975202]
